## SparseMatrix
1. SparseMatrix 클래스 생성
2. 생성자로 dimension (m, n)값을 받고, 리스트를 [[m, n, 0]]으로 초기 생성한다.
3. append method : (row, col, value)을 받고, 리스트에 추가한다.
4. shape method : 희소행렬 dimension 리턴
5. getValue method : (row, col) 입력하면, value값 리턴
6. print method : 희소행렬을 일반적 행렬 형태로 프린트 (numpy.zeros() 이용해서 0으로 구성된 m,n array 만들고 row, col값을 업데이트 한다.
7. add classmethod : 우선 두 행렬이 같은 dimension 가지고 있는지 확인. (불일치하면 "dimenstion error"를 리턴) 이후, 새로운 객체 생성해서, set에 non-zero value들의 row, col값들을 넣고 중복을 없앤다. 그리고 남아있는 row, col들의 value들을 합해서 새로운 객체에 넣는다. 이 때, 더해진 value가 0이 아닌 것들만 append 한다.
8. mult classmethod : A(r*c), B(c*l) -> A의 열 수와 B의 행 수가 같아야 한다. 결과는 (r*l) dimension을 가지게 된다. (각각의 A의 행과 B의 열끼리 곱해져 전부 더해지게 된다.) 이 때도, value가 0이 아닌 경우에만 append 한다.


In [25]:
import numpy as np
class SparseMatrix:
    
    def __init__(self, m, n):
        self.m = m
        self.n = n
        self.s = [[m, n, 0]]

    def append(self, row, col, value):
        if value != 0:
            self.s.append([row, col, value])
        self.s[0][2] = len(self.s) - 1
        # print(self.s)

    def shape(self):
        return (self.s[0][0], self.s[0][1])

    def getValue(self, row, col):
        for i in range(1, len(self.s)):
            if self.s[i][0] == row and self.s[i][1] == col:
                return self.s[i][2]
        return 0

    def print(self):
        _tmp = np.zeros((self.m, self.n))
        for i in range(1, len(self.s)):
            _tmp[(self.s[i][0] - 1), self.s[i][1] - 1] = self.s[i][2]
        print(_tmp)
    
    @classmethod
    def add(cls, p, q):
        if p.m != q.m or p.n != q.n:
            return "dimension is not match"

        r = SparseMatrix(p.m, p.n)
        u = set()

        for i in range(1, len(p.s)):
            u.add((p.s[i][0], p.s[i][1]))
        for i in range(1, len(q.s)):
            u.add((q.s[i][0], q.s[i][1]))
        # print(u)

        for tup in list(u):
            _tmp = p.getValue(tup[0], tup[1]) + q.getValue(tup[0], tup[1])
            if _tmp != 0:
                r.append(tup[0], tup[1], _tmp)
        return r

    @classmethod
    def mult(cls, p, q):
        if p.shape()[1] != q.shape()[0]:
            return 'dimension error'
        else:
            m = p.shape()[0]
            n = q.shape()[1]
            l = p.shape()[1]

        r = SparseMatrix(m, n)
        for i in range(1, m+1):
            for j in range(1, n+1):
                sum = 0
                for k in range(1, l+1):
                    sum += p.getValue(i, k) * q.getValue(k, j)
                if sum != 0:
                    r.append(i, j, sum)

        return r 


a = SparseMatrix(3, 3)
a.append(1, 1, 3)
a.append(1, 3, 0)
a.append(2, 2, 1)
a.append(3, 3, 3)

b = SparseMatrix(3, 3)
b.append(1, 1, 4)
b.append(1, 2, 2)
b.append(2, 1, 6)
b.append(3, 3, 5)

# a.print()
# b.print()
# c = SparseMatrix.add(a, b)
# c.print()

d = SparseMatrix.mult(a, b)

a.print()
b.print()
d.print()

[[3. 0. 0.]
 [0. 1. 0.]
 [0. 0. 3.]]
[[4. 2. 0.]
 [6. 0. 0.]
 [0. 0. 5.]]
[[12.  6.  0.]
 [ 6.  0.  0.]
 [ 0.  0. 15.]]
